In [195]:
import pandas as pd
import glob
import json
import requests
from pprint import pprint

# This package makes it easier to work with United States data
from us import states # https://github.com/unitedstates/python-us

# This package allows you to read Census data in Python
from census import Census # https://github.com/datamade/census

In [227]:
# Sign up for a CENSUS API Key
# https://api.census.gov/data/key_signup.html

c = Census("YOUR-API-KEY-HERE")

# CensusReporter

Check out https://censusreporter.org/ to learn more about how variables are collected by the US Census. 

👉 Which table number do we need to get the total population?

## Using the Census Package

Let's get data from table B01003

In [197]:
TABLE = 'B01003' #population
tables = [t for t in c.acs5.tables() if 'B01003' in t['name']]
tables

[{'name': 'B01003',
  'description': 'TOTAL POPULATION',
  'variables': 'http://api.census.gov/data/2019/acs/acs5/groups/B01003.json'}]

Hmm...interesting, there is the table and the description. What happens if I go and grab that JSON file?

In [198]:
url = tables[0]['variables']
variables = requests.get(url).json()
variables

{'variables': {'B01003_001E': {'label': 'Estimate!!Total',
   'concept': 'TOTAL POPULATION',
   'predicateType': 'int',
   'group': 'B01003',
   'limit': 0,
   'predicateOnly': True},
  'B01003_001M': {'label': 'Margin of Error!!Total',
   'concept': 'TOTAL POPULATION',
   'predicateType': 'int',
   'group': 'B01003',
   'limit': 0,
   'predicateOnly': True},
  'B01003_001MA': {'label': 'Annotation of Margin of Error!!Total',
   'concept': 'TOTAL POPULATION',
   'predicateType': 'string',
   'group': 'B01003',
   'limit': 0,
   'predicateOnly': True},
  'B01003_001EA': {'label': 'Annotation of Estimate!!Total',
   'concept': 'TOTAL POPULATION',
   'predicateType': 'string',
   'group': 'B01003',
   'limit': 0,
   'predicateOnly': True}}}

Aha, we have a schema! Let's take a closer look at the keys and their labels.

In [199]:
variable_labels= {}
for key, value in variables['variables'].items():
    variable_labels[key] = value['label']
    
pprint(variable_labels)

{'B01003_001E': 'Estimate!!Total',
 'B01003_001EA': 'Annotation of Estimate!!Total',
 'B01003_001M': 'Margin of Error!!Total',
 'B01003_001MA': 'Annotation of Margin of Error!!Total'}


In [200]:
variable_list = list(variable_labels.keys())
pprint(variable_list)

['B01003_001E', 'B01003_001M', 'B01003_001MA', 'B01003_001EA']


Now let's go get the data for a particular geography (we'll do congressional districts for now). You can check the datamade census package documentation to see how this works. https://github.com/datamade/census

In [201]:
populations_by_congressional_district = c.acs5.state_congressional_district(
    ['NAME'] + variable_list,
    state_fips = Census.ALL,
    congressional_district = Census.ALL
)

## An alternate way to query the same data.

# populations_by_congressional_district = c.acs5.get(['NAME'] + variable_list, geo={
#     'for': 'congressional district:*'
# })

populations_by_congressional_district

[{'NAME': 'Congressional District 10 (116th Congress), Florida',
  'B01003_001E': 823865.0,
  'B01003_001M': 4891.0,
  'B01003_001MA': None,
  'B01003_001EA': None,
  'state': '12',
  'congressional district': '10'},
 {'NAME': 'Congressional District 4 (116th Congress), Florida',
  'B01003_001E': 800945.0,
  'B01003_001M': 5003.0,
  'B01003_001MA': None,
  'B01003_001EA': None,
  'state': '12',
  'congressional district': '04'},
 {'NAME': 'Congressional District 13 (116th Congress), Florida',
  'B01003_001E': 729991.0,
  'B01003_001M': 2461.0,
  'B01003_001MA': None,
  'B01003_001EA': None,
  'state': '12',
  'congressional district': '13'},
 {'NAME': 'Congressional District 26 (116th Congress), Florida',
  'B01003_001E': 768910.0,
  'B01003_001M': 4632.0,
  'B01003_001MA': None,
  'B01003_001EA': None,
  'state': '12',
  'congressional district': '26'},
 {'NAME': 'Congressional District 22 (116th Congress), Florida',
  'B01003_001E': 755091.0,
  'B01003_001M': 3939.0,
  'B01003_001MA'

Let's turn that into a dataframe

In [202]:
population_df = pd.DataFrame(populations_by_congressional_district)
population_df

,NAME,B01003_001E,B01003_001M,B01003_001MA,B01003_001EA,state,congressional district
0,"Congressional District 10 (116th Congress), Fl...",823865.0,4891.0,None,None,12,10
1,"Congressional District 4 (116th Congress), Flo...",800945.0,5003.0,None,None,12,04
2,"Congressional District 13 (116th Congress), Fl...",729991.0,2461.0,None,None,12,13
3,"Congressional District 26 (116th Congress), Fl...",768910.0,4632.0,None,None,12,26
4,"Congressional District 22 (116th Congress), Fl...",755091.0,3939.0,None,None,12,22
...,...,...,...,...,...,...,...
435,"Congressional District 11 (116th Congress), Mi...",730595.0,745.0,None,None,26,11
436,"Congressional District 8 (116th Congress), Mic...",739522.0,454.0,None,None,26,08
437,"Congressional District 2 (116th Congress), Min...",703966.0,424.0,None,None,27,02
438,"Congressional District 1 (116th Congress), Min...",675187.0,304.0,None,None,27,01


And add some useful columns using the united states package to look up the state from the fips code
https://github.com/unitedstates/python-us

In [203]:
population_df['state_fips'] = population_df['state']
population_df['state'] = population_df.state.apply(lambda x: states.lookup(x))
population_df['state_abbrev'] = population_df.state.apply(lambda x: x.abbr)
population_df

,NAME,B01003_001E,B01003_001M,B01003_001MA,B01003_001EA,state,congressional district,state_fips,state_abbrev
0,"Congressional District 10 (116th Congress), Fl...",823865.0,4891.0,None,None,Florida,10,12,FL
1,"Congressional District 4 (116th Congress), Flo...",800945.0,5003.0,None,None,Florida,04,12,FL
2,"Congressional District 13 (116th Congress), Fl...",729991.0,2461.0,None,None,Florida,13,12,FL
3,"Congressional District 26 (116th Congress), Fl...",768910.0,4632.0,None,None,Florida,26,12,FL
4,"Congressional District 22 (116th Congress), Fl...",755091.0,3939.0,None,None,Florida,22,12,FL
...,...,...,...,...,...,...,...,...,...
435,"Congressional District 11 (116th Congress), Mi...",730595.0,745.0,None,None,Michigan,11,26,MI
436,"Congressional District 8 (116th Congress), Mic...",739522.0,454.0,None,None,Michigan,08,26,MI
437,"Congressional District 2 (116th Congress), Min...",703966.0,424.0,None,None,Minnesota,02,27,MN
438,"Congressional District 1 (116th Congress), Min...",675187.0,304.0,None,None,Minnesota,01,27,MN


Finally, I'll rename the variables according to their labels

In [204]:
population_df = population_df.rename(columns = variable_labels)
population_df

,NAME,Estimate!!Total,Margin of Error!!Total,Annotation of Margin of Error!!Total,Annotation of Estimate!!Total,state,congressional district,state_fips,state_abbrev
0,"Congressional District 10 (116th Congress), Fl...",823865.0,4891.0,None,None,Florida,10,12,FL
1,"Congressional District 4 (116th Congress), Flo...",800945.0,5003.0,None,None,Florida,04,12,FL
2,"Congressional District 13 (116th Congress), Fl...",729991.0,2461.0,None,None,Florida,13,12,FL
3,"Congressional District 26 (116th Congress), Fl...",768910.0,4632.0,None,None,Florida,26,12,FL
4,"Congressional District 22 (116th Congress), Fl...",755091.0,3939.0,None,None,Florida,22,12,FL
...,...,...,...,...,...,...,...,...,...
435,"Congressional District 11 (116th Congress), Mi...",730595.0,745.0,None,None,Michigan,11,26,MI
436,"Congressional District 8 (116th Congress), Mic...",739522.0,454.0,None,None,Michigan,08,26,MI
437,"Congressional District 2 (116th Congress), Min...",703966.0,424.0,None,None,Minnesota,02,27,MN
438,"Congressional District 1 (116th Congress), Min...",675187.0,304.0,None,None,Minnesota,01,27,MN


🎉 Tada! We got some data from the Census! 

### Let's get some data on the racial makeup of congressional districts now.
This is following the same process as above.

In [205]:
RACE_TABLE = 'B03002' # Acquired this after looking through censusreporter.org (at the bottom of the page)
tables = [t for t in c.acs5.tables() if RACE_TABLE in t['name']]
tables

[{'name': 'B03002',
  'description': 'HISPANIC OR LATINO ORIGIN BY RACE',
  'variables': 'http://api.census.gov/data/2019/acs/acs5/groups/B03002.json'}]

In [206]:
labels = requests.get(tables[0]['variables']).json()['variables']
labels = { k:labels[k]['label'] for k in labels.keys()}

# looked through the json above and picked out the variables I wanted
race_variables = ['B01003_001E','B03002_012E','B03002_006E','B03002_005E',
                  'B03002_004E','B03002_003E','B03002_007E','B03002_008E','B03002_009E',]


In [207]:
# query the data 
population_by_race = pd.DataFrame(c.acs5.get(['NAME'] + race_variables, geo={
    'for': 'congressional district:*'
}))

# make state columns more useful
population_by_race['state_fips'] = population_by_race['state']
population_by_race['state'] = population_by_race.state.apply(lambda x: states.lookup(x))
population_by_race['state_abbrev'] = population_by_race.state.apply(lambda x: x.abbr)


# rename by labels
population_by_race = population_by_race.rename(columns=labels)

population_by_race.head(2)

,NAME,B01003_001E,Estimate!!Total:!!Hispanic or Latino:,Estimate!!Total:!!Not Hispanic or Latino:!!Asian alone,Estimate!!Total:!!Not Hispanic or Latino:!!American Indian and Alaska Native alone,Estimate!!Total:!!Not Hispanic or Latino:!!Black or African American alone,Estimate!!Total:!!Not Hispanic or Latino:!!White alone,Estimate!!Total:!!Not Hispanic or Latino:!!Native Hawaiian and Other Pacific Islander alone,Estimate!!Total:!!Not Hispanic or Latino:!!Some other race alone,Estimate!!Total:!!Not Hispanic or Latino:!!Two or more races:,state,congressional district,state_fips,state_abbrev
0,"Congressional District 10 (116th Congress), Fl...",823865.0,233746.0,41819.0,969.0,221852.0,296087.0,457.0,8767.0,20168.0,Florida,10,12,FL
1,"Congressional District 4 (116th Congress), Flo...",800945.0,68161.0,35819.0,1997.0,73354.0,595852.0,309.0,3240.0,22213.0,Florida,04,12,FL


In [208]:
# Some more cleanup
population_by_race.columns = [x.replace('Estimate!!Total:!!', '') for x in population_by_race.columns]
population_by_race.columns = [x.replace('Not Hispanic or Latino:!!', '') for x in population_by_race.columns]
population_by_race.columns = [x.replace(':', '') for x in population_by_race.columns]
population_by_race.columns = [x.replace('alone', '') for x in population_by_race.columns]
population_by_race.columns = [x.replace('B01003_001E', 'Total Population') for x in population_by_race.columns]
population_by_race.head(2)

,NAME,Total Population,Hispanic or Latino,Asian,American Indian and Alaska Native,Black or African American,White,Native Hawaiian and Other Pacific Islander,Some other race,Two or more races,state,congressional district,state_fips,state_abbrev
0,"Congressional District 10 (116th Congress), Fl...",823865.0,233746.0,41819.0,969.0,221852.0,296087.0,457.0,8767.0,20168.0,Florida,10,12,FL
1,"Congressional District 4 (116th Congress), Flo...",800945.0,68161.0,35819.0,1997.0,73354.0,595852.0,309.0,3240.0,22213.0,Florida,04,12,FL


In [209]:
race_variables = ['Hispanic or Latino', 
 'Asian ', 
 'American Indian and Alaska Native ', 
 'Black or African American ',
 'White ',
 'Native Hawaiian and Other Pacific Islander ',
 'Some other race ', 'Two or more races']

# convert to percents
for race in race_variables:
    new_key = 'pct_' + race.lower().strip().replace(' ', '_')
    population_by_race[new_key] = (population_by_race[race] / population_by_race['Total Population'] * 100).round(2)
    population_by_race = population_by_race.drop(columns=race)
    
display(population_by_race)

,NAME,Total Population,state,congressional district,state_fips,state_abbrev,pct_hispanic_or_latino,pct_asian,pct_american_indian_and_alaska_native,pct_black_or_african_american,pct_white,pct_native_hawaiian_and_other_pacific_islander,pct_some_other_race,pct_two_or_more_races
0,"Congressional District 10 (116th Congress), Fl...",823865.0,Florida,10,12,FL,28.37,5.08,0.12,26.93,35.94,0.06,1.06,2.45
1,"Congressional District 4 (116th Congress), Flo...",800945.0,Florida,04,12,FL,8.51,4.47,0.25,9.16,74.39,0.04,0.40,2.77
2,"Congressional District 13 (116th Congress), Fl...",729991.0,Florida,13,12,FL,10.24,3.50,0.22,12.07,71.10,0.09,0.30,2.48
3,"Congressional District 26 (116th Congress), Fl...",768910.0,Florida,26,12,FL,70.93,1.72,0.08,10.19,16.01,0.02,0.30,0.74
4,"Congressional District 22 (116th Congress), Fl...",755091.0,Florida,22,12,FL,22.44,3.16,0.12,14.65,57.16,0.03,0.61,1.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,"Congressional District 11 (116th Congress), Mi...",730595.0,Michigan,11,26,MI,3.42,10.57,0.20,5.05,78.22,0.02,0.15,2.38
436,"Congressional District 8 (116th Congress), Mic...",739522.0,Michigan,08,26,MI,5.21,4.73,0.21,5.44,81.63,0.02,0.10,2.67
437,"Congressional District 2 (116th Congress), Min...",703966.0,Minnesota,02,27,MN,6.21,4.74,0.34,4.94,80.79,0.03,0.19,2.75
438,"Congressional District 1 (116th Congress), Min...",675187.0,Minnesota,01,27,MN,6.54,2.77,0.25,3.31,85.29,0.07,0.08,1.69


# Add poverty levels

In [210]:
POVERTY_TABLE = 'B17001'  # Acquired this after looking through censusreporter.org (at the bottom of the page)

tables = [t for t in c.acs5.tables() if POVERTY_TABLE == t['name']]
tables

[{'name': 'B17001',
  'description': 'POVERTY STATUS IN THE PAST 12 MONTHS BY SEX BY AGE',
  'variables': 'http://api.census.gov/data/2019/acs/acs5/groups/B17001.json'}]

In [211]:
variables = requests.get(f"http://api.census.gov/data/2019/acs/acs5/groups/{POVERTY_TABLE}.json").json()
variables = variables['variables']
variables = ['B17001_001E', 'B17001_002E']

In [212]:
population_by_poverty = pd.DataFrame(
    c.acs5.state_congressional_district(
    fields = ['NAME'] + variables,
    state_fips = Census.ALL, 
    year = 2019,
    congressional_district = Census.ALL,
    table=[POVERTY_TABLE])
)

# convert some columns
population_by_poverty['state_fips'] = population_by_poverty['state']
population_by_poverty['state'] = population_by_poverty.state.apply(lambda x: states.lookup(x))
population_by_poverty['state_abbrev'] = population_by_poverty.state.apply(lambda x: x.abbr)

# display
display(population_by_poverty.head())

# calculate percent below poverty
population_by_poverty['pct_below_poverty'] = (population_by_poverty['B17001_002E'] / population_by_poverty['B17001_001E'] * 100).round(2)

# drop columns I don't need
population_by_poverty = population_by_poverty[['NAME', 'state', 'state_abbrev', 'congressional district', 'pct_below_poverty']]

# display
population_by_poverty

,NAME,B17001_001E,B17001_002E,state,congressional district,state_fips,state_abbrev
0,"Congressional District 10 (116th Congress), Fl...",814009.0,122350.0,Florida,10,12,FL
1,"Congressional District 4 (116th Congress), Flo...",783822.0,67953.0,Florida,04,12,FL
2,"Congressional District 13 (116th Congress), Fl...",717495.0,95318.0,Florida,13,12,FL
3,"Congressional District 26 (116th Congress), Fl...",756670.0,115221.0,Florida,26,12,FL
4,"Congressional District 22 (116th Congress), Fl...",746061.0,83187.0,Florida,22,12,FL


,NAME,state,state_abbrev,congressional district,pct_below_poverty
0,"Congressional District 10 (116th Congress), Fl...",Florida,FL,10,15.03
1,"Congressional District 4 (116th Congress), Flo...",Florida,FL,04,8.67
2,"Congressional District 13 (116th Congress), Fl...",Florida,FL,13,13.28
3,"Congressional District 26 (116th Congress), Fl...",Florida,FL,26,15.23
4,"Congressional District 22 (116th Congress), Fl...",Florida,FL,22,11.15
...,...,...,...,...,...
435,"Congressional District 11 (116th Congress), Mi...",Michigan,MI,11,6.01
436,"Congressional District 8 (116th Congress), Mic...",Michigan,MI,08,10.85
437,"Congressional District 2 (116th Congress), Min...",Minnesota,MN,02,6.04
438,"Congressional District 1 (116th Congress), Min...",Minnesota,MN,01,10.65


## Combine

In [224]:
population_by_race

,NAME,Total Population,state,congressional district,state_fips,state_abbrev,pct_hispanic_or_latino,pct_asian,pct_american_indian_and_alaska_native,pct_black_or_african_american,pct_white,pct_native_hawaiian_and_other_pacific_islander,pct_some_other_race,pct_two_or_more_races
0,"Congressional District 10 (116th Congress), Fl...",823865.0,Florida,10,12,FL,28.37,5.08,0.12,26.93,35.94,0.06,1.06,2.45
1,"Congressional District 4 (116th Congress), Flo...",800945.0,Florida,04,12,FL,8.51,4.47,0.25,9.16,74.39,0.04,0.40,2.77
2,"Congressional District 13 (116th Congress), Fl...",729991.0,Florida,13,12,FL,10.24,3.50,0.22,12.07,71.10,0.09,0.30,2.48
3,"Congressional District 26 (116th Congress), Fl...",768910.0,Florida,26,12,FL,70.93,1.72,0.08,10.19,16.01,0.02,0.30,0.74
4,"Congressional District 22 (116th Congress), Fl...",755091.0,Florida,22,12,FL,22.44,3.16,0.12,14.65,57.16,0.03,0.61,1.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,"Congressional District 11 (116th Congress), Mi...",730595.0,Michigan,11,26,MI,3.42,10.57,0.20,5.05,78.22,0.02,0.15,2.38
436,"Congressional District 8 (116th Congress), Mic...",739522.0,Michigan,08,26,MI,5.21,4.73,0.21,5.44,81.63,0.02,0.10,2.67
437,"Congressional District 2 (116th Congress), Min...",703966.0,Minnesota,02,27,MN,6.21,4.74,0.34,4.94,80.79,0.03,0.19,2.75
438,"Congressional District 1 (116th Congress), Min...",675187.0,Minnesota,01,27,MN,6.54,2.77,0.25,3.31,85.29,0.07,0.08,1.69


In [223]:
population_by_poverty

,NAME,state,state_abbrev,congressional district,pct_below_poverty
0,"Congressional District 10 (116th Congress), Fl...",Florida,FL,10,15.03
1,"Congressional District 4 (116th Congress), Flo...",Florida,FL,04,8.67
2,"Congressional District 13 (116th Congress), Fl...",Florida,FL,13,13.28
3,"Congressional District 26 (116th Congress), Fl...",Florida,FL,26,15.23
4,"Congressional District 22 (116th Congress), Fl...",Florida,FL,22,11.15
...,...,...,...,...,...
435,"Congressional District 11 (116th Congress), Mi...",Michigan,MI,11,6.01
436,"Congressional District 8 (116th Congress), Mic...",Michigan,MI,08,10.85
437,"Congressional District 2 (116th Congress), Min...",Minnesota,MN,02,6.04
438,"Congressional District 1 (116th Congress), Min...",Minnesota,MN,01,10.65


In [226]:
census_df = population_by_race.merge(population_by_poverty, on=['NAME', 'state_abbrev', 'state', 'congressional district'])
census_df.to_csv('census_df.csv', index=False)
census_df

,NAME,Total Population,state,congressional district,state_fips,state_abbrev,pct_hispanic_or_latino,pct_asian,pct_american_indian_and_alaska_native,pct_black_or_african_american,pct_white,pct_native_hawaiian_and_other_pacific_islander,pct_some_other_race,pct_two_or_more_races,pct_below_poverty
0,"Congressional District 10 (116th Congress), Fl...",823865.0,Florida,10,12,FL,28.37,5.08,0.12,26.93,35.94,0.06,1.06,2.45,15.03
1,"Congressional District 4 (116th Congress), Flo...",800945.0,Florida,04,12,FL,8.51,4.47,0.25,9.16,74.39,0.04,0.40,2.77,8.67
2,"Congressional District 13 (116th Congress), Fl...",729991.0,Florida,13,12,FL,10.24,3.50,0.22,12.07,71.10,0.09,0.30,2.48,13.28
3,"Congressional District 26 (116th Congress), Fl...",768910.0,Florida,26,12,FL,70.93,1.72,0.08,10.19,16.01,0.02,0.30,0.74,15.23
4,"Congressional District 22 (116th Congress), Fl...",755091.0,Florida,22,12,FL,22.44,3.16,0.12,14.65,57.16,0.03,0.61,1.81,11.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,"Congressional District 11 (116th Congress), Mi...",730595.0,Michigan,11,26,MI,3.42,10.57,0.20,5.05,78.22,0.02,0.15,2.38,6.01
436,"Congressional District 8 (116th Congress), Mic...",739522.0,Michigan,08,26,MI,5.21,4.73,0.21,5.44,81.63,0.02,0.10,2.67,10.85
437,"Congressional District 2 (116th Congress), Min...",703966.0,Minnesota,02,27,MN,6.21,4.74,0.34,4.94,80.79,0.03,0.19,2.75,6.04
438,"Congressional District 1 (116th Congress), Min...",675187.0,Minnesota,01,27,MN,6.54,2.77,0.25,3.31,85.29,0.07,0.08,1.69,10.65
